In [1]:
!pip install -qU langchain langchain_groq langchain_community
!pip install -qU duckduckgo-search arxiv wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.4 MB/s eta 0:00:00a 0:00:01


# LLM Model

### Groq Model

In [ ]:
GROQ_API_KEY = ""

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    groq_api_key=GROQ_API_KEY,
    temperature=0
)

# Prompt Template

In [4]:
from langchain import PromptTemplate

# The ReAct template for Mixtral Model
react_template = """
<s>[INST] You are an AI assistant that answers questions by breaking them down and using tools precisely.
Answer the following questions as best you can. You have access to the following tools:
{tools}

Previous conversation history:
{chat_history}

Use the following format STRICTLY and EXACTLY:
Question: the input question you must answer  
Thought: you should always think about what to do  
Action: the action to take, should be one of [{tool_names}]  
Action Input: the input to the action  
Observation: the result of the action  
... (this Thought/Action/Action Input/Observation can repeat N times)  
Thought: I now know the final answer  
Final Answer: the final answer to the original input question  

IMPORTANT: After providing the Final Answer, you must STOP. Do not generate any additional text or steps.
IMPORTANT: If a tool result is bad, try another tool.
IMPORTANT: Use the chat history to provide more contextual and consistent responses.

Question: {input}
{agent_scratchpad} [/INST]
Thought: Let me break this down step by step and follow the format exactly...
{agent_scratchpad}</s>
"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad", "chat_history"]
)

# Tools

In [5]:
from langchain.agents import Tool, load_tools 
from langchain.tools import DuckDuckGoSearchResults
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

In [6]:
duckduck = DuckDuckGoSearchResults()
duckduck_tool = Tool(
    name="DuckDuckGoSearch",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=duckduck.run)

wiki_wrapper = WikipediaAPIWrapper()
wiki_query = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool = Tool(
    name="Wikipedia Search Tool",
    description="An API search tool, use it to retrieve information from Wikipedia articles.",
    func=wiki_query.run)

arxiv_wrapper = ArxivAPIWrapper()
arxiv_query = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool = Tool(
    name="Arxiv Search Tool",
    description="An API search tool, use it to search for academic papers, preprints, and research articles on Arxiv.",
    func=arxiv_query.run)

tools = [duckduck_tool, wiki_tool, arxiv_tool]
print(tools)

[Tool(name='DuckDuckGoSearch', description='A web search engine. Use this to as a search engine for general queries.', func=<bound method BaseTool.run of DuckDuckGoSearchResults(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))>), Tool(name='Wikipedia Search Tool', description='An API search tool, use it to retrieve information from Wikipedia articles.', func=<bound method BaseTool.run of WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))>), Tool(name='Arxiv Search Tool', description='An API search tool, use it to search for academic papers, preprints, and research articles on Arxiv.', func=<bound method BaseTool.run of ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(

# Memory

In [7]:
from langchain.memory import ConversationBufferMemory

buffer_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="output"
)

<ipython-input-7-2190517d13ab>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory(


# Agent

In [8]:
from langchain.agents import AgentExecutor,create_react_agent

agent = create_react_agent(llm, tools, prompt)

# Create the executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=buffer_memory,
    verbose=False,
    handle_parsing_errors=True,
    return_intermediate_steps=False
)

In [9]:
def chat_with_memory(question):
    if "clear memory" in question.lower() or "reset" in question.lower():
        buffer_memory.chat_memory.clear()
        return "Memory has been cleared!"
    
    result = agent_executor.invoke(
        {
            "input": question,
            "chat_history": buffer_memory.chat_memory.messages
        }
    )
    return result["output"]

# Chatbot

In [10]:
def start_chatbot():
    print("Hello! I am your assistant. You can start asking questions now.\n(Type 'exit' to end the chat.)")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() in ["exit", "quit", "q"]:
            print("Goodbye! Have a great day!")
            break
        
        result = chat_with_memory(user_input)
        print("Bot:", result)

        # Display chat history
        # print("Current Memory:")
        # print(buffer_memory.chat_memory.messages)

# Run the chatbot
start_chatbot()

Hello! I am your assistant. You can start asking questions now.
(Type 'exit' to end the chat.)


You:  Hi, How are you?


Bot: Hello! I'm an AI assistant, and I'm here to help you with any questions you have. How can I assist you today?

(Stop)


You:  who is the founder of spaceX?


Bot: Elon Musk is the founder of SpaceX.


...

Bot: Elon Musk is associated with several companies, including PayPal, SpaceX, Tesla, Neuralink, and The Boring Company.


You:  clear memory


Bot: Memory has been cleared!


You:  exit


Goodbye! Have a great day!
